### University Information System APIs

In [1]:
from flask import Flask, request, jsonify
import mysql.connector

app = Flask(__name__)

In [2]:
# Database configuration
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "marvel99",
    "database": "dav6100s25"
}

In [3]:
# Helper function for database connection
def get_db_connection():
    return mysql.connector.connect(**db_config)

# Helper function for pagination
def paginate_query(query, params, page, page_size):
    offset = (page - 1) * page_size
    paginated_query = query + " LIMIT %s OFFSET %s"
    return paginated_query, params + [page_size, offset]

In [4]:
# Departments API
@app.route('/departments', methods=['GET'])
def get_departments():
    try:
        page = int(request.args.get('page', 1))
        page_size = int(request.args.get('page_size', 10))
        
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        
        # Query for department and instructor information
        query = """
            SELECT d.dept_name, d.building, d.budget, 
                   i.ID AS instructor_id, i.name AS instructor_name, i.salary
            FROM department d
            LEFT JOIN instructor i ON d.dept_name = i.dept_name
        """
        paginated_query, params = paginate_query(query, [], page, page_size)
        cursor.execute(paginated_query, params)
        records = cursor.fetchall()

        # Group instructors by department
        department_data = {}
        for record in records:
            dept_name = record["dept_name"]
            if dept_name not in department_data:
                department_data[dept_name] = {
                    "dept_name": record["dept_name"],
                    "building": record["building"],
                    "budget": record["budget"],
                    "instructors": []
                }
            if record["instructor_id"]:
                department_data[dept_name]["instructors"].append({
                    "id": record["instructor_id"],
                    "name": record["instructor_name"],
                    "salary": record["salary"]
                })

        # Count total records
        cursor.execute("SELECT COUNT(*) AS total FROM department")
        total = cursor.fetchone()["total"]

        conn.close()
        
        return jsonify({
            "code": 1,
            "msg": "Success",
            "data": {
                "total": total,
                "records": list(department_data.values())
            }
        })

    except Exception as e:
        return jsonify({"code": 0, "msg": "Error", "data": str(e)})

In [5]:
# Students API
@app.route('/students', methods=['GET'])
def get_students():
    try:
        page = int(request.args.get('page', 1))
        page_size = int(request.args.get('page_size', 10))
        
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        
        # Query for student and course enrollment information
        query = """
            SELECT s.ID AS student_id, s.name AS student_name, s.dept_name, 
                   s.tot_cred, t.course_id, t.sec_id, t.semester, t.year
            FROM student s
            LEFT JOIN takes t ON s.ID = t.ID
        """
        paginated_query, params = paginate_query(query, [], page, page_size)
        cursor.execute(paginated_query, params)
        records = cursor.fetchall()

        # Group courses by student
        student_data = {}
        for record in records:
            student_id = record["student_id"]
            if student_id not in student_data:
                student_data[student_id] = {
                    "id": record["student_id"],
                    "name": record["student_name"],
                    "dept_name": record["dept_name"],
                    "courses": []
                }
            if record["course_id"]:
                student_data[student_id]["courses"].append({
                    "course_id": record["course_id"],
                    "section_id": record["sec_id"],
                    "semester": record["semester"],
                    "year": record["year"]
                })

        # Count total records
        cursor.execute("SELECT COUNT(*) AS total FROM student")
        total = cursor.fetchone()["total"]

        conn.close()
        
        return jsonify({
            "code": 1,
            "msg": "Success",
            "data": {
                "total": total,
                "records": list(student_data.values())
            }
        })

    except Exception as e:
        return jsonify({"code": 0, "msg": "Error", "data": str(e)})

In [6]:
# Courses API
@app.route('/courses', methods=['GET'])
def get_courses():
    try:
        page = int(request.args.get('page', 1))
        page_size = int(request.args.get('page_size', 10))
        
        conn = get_db_connection()
        cursor = conn.cursor(dictionary=True)
        
        # Query for course and instructor information
        query = """
            SELECT c.course_id, c.title, c.dept_name,
                   t.ID AS instructor_id, i.name AS instructor_name,
                   t.sec_id, t.semester, t.year
            FROM course c
            LEFT JOIN teaches t ON c.course_id = t.course_id
            LEFT JOIN instructor i ON t.ID = i.ID
        """
        paginated_query, params = paginate_query(query, [], page, page_size)
        cursor.execute(paginated_query, params)
        records = cursor.fetchall()

        # Group instructors by course
        course_data = {}
        for record in records:
            course_id = record["course_id"]
            if course_id not in course_data:
                course_data[course_id] = {
                    "course_id": record["course_id"],
                    "title": record["title"],
                    "dept_name": record["dept_name"],
                    "instructors": []
                }
            if record["instructor_id"]:
                course_data[course_id]["instructors"].append({
                    "instructor_id": record["instructor_id"],
                    "name": record["instructor_name"],
                    "section": record["sec_id"],
                    "semester": record["semester"],
                    "year": record["year"]
                })

        # Count total records
        cursor.execute("SELECT COUNT(*) AS total FROM course")
        total = cursor.fetchone()["total"]

        conn.close()
        
        return jsonify({
            "code": 1,
            "msg": "Success",
            "data": {
                "total": total,
                "records": list(course_data.values())
            }
        })

    except Exception as e:
        return jsonify({"code": 0, "msg": "Error", "data": str(e)})


In [ ]:
# Run the application
if __name__ == '__main__':
    app.run(port=5050)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5050
Press CTRL+C to quit
127.0.0.1 - - [13/May/2025 18:50:43] "GET /departments HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 18:50:43] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/May/2025 19:04:44] "GET /departments?page=1&page_size=10 HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 19:04:44] "GET /departments?page=2&page_size=10 HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 19:04:44] "GET /departments?page=3&page_size=10 HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 19:04:44] "GET /students?page=1&page_size=10 HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 19:04:44] "GET /students?page=2&page_size=10 HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 19:04:44] "GET /students?page=3&page_size=10 HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 19:04:44] "GET /students?page=4&page_size=10 HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 19:04:44] "GET /students?page=5&page_size=10 HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2025 19:04:44] "GET /courses?page=1&page_size=10 HTTP/1.1" 200 -
1

In [ ]:
# HTML homepage 
html_content_updated = """
<!DOCTYPE html>
<html>
<head>
    <title>University API Homepage</title>
</head>
<body>
    <h1>Welcome to the University Information System API</h1>
    <p>This web service provides access to university-related data via three RESTful API endpoints:</p>
    <ul>
        <li><a href="http://127.0.0.1:5050/departments?page=1&page_size=10">/departments</a> - View department info and their instructors</li>
        <li><a href="http://127.0.0.1:5050/students?page=1&page_size=10">/students</a> - View students and their enrolled courses</li>
        <li><a href="http://127.0.0.1:5050/courses?page=1&page_size=10">/courses</a> - View course offerings and teaching faculty</li>
    </ul>
</body>
</html>
"""

with open("static/index.html", "w") as file:
    file.write(html_content_updated)

